# Introduction to the Pulse Sequence: Exciting an atom in the Rydberg state

In [ ]:
import numpy as np
import pulser
from pprint import pprint
import pulser_simulation

The `Sequence` is the core object in Pulser. It enables the manipulation of the state of the atoms by controlling their configuration in the `Register` and the application of `Pulses`. This tutorial illustrates the [Conventions page](https://pulser.readthedocs.io/en/stable/conventions.html). It shows the step-by-step creation of a simple Pulser Sequence that excites one atom to the Rydberg state, and how to simulate it using a backend.

## 1. Creating the `Register`

The `Register` defines the positions of the atoms and their associated names. There are multiple ways of defining a `Register`, and they are further described in the [Register section](https://pulser.readthedocs.io/en/stable/tutorials/reg_layouts.html) of the fundamentals.

Let's create a `Register` composed of only 1 atom placed at the coordinate `(0, 0)`, using the `Register.from_coordinates` class method: 

In [ ]:
reg = pulser.Register.from_coordinates([(0, 0)], prefix="q")
reg.draw()

## 2. Initializing the Sequence

To create a `Sequence`, one has to provide it with the `Register` instance and the `Device` in which the sequence will be executed. A `Device` object stores all the physical constraints the `Sequence` has to match. It will first check that the `Register` is valid, and will be used to validate each `Pulse` that will be added to the `Sequence`. For more information regarding the `Device`, you can check [its section in the Fundamentals](https://pulser.readthedocs.io/en/stable/tutorials/virtual_devices.html).

Pulser provides some examples of `Device` in `pulser.devices` (find the list of these examples [in the API](https://pulser.readthedocs.io/en/stable/apidoc/core.html#physical-devices)).

Each QPU has an associated `Device`. The available devices can be fetched using a `RemoteConnection`. Have a look to [the page on QPU Backend](https://pulser.readthedocs.io/en/stable/tutorials/backends.html) to see how to fetch the `Device` associated with the QPU.

Let's build our `Sequence` with the `Register` previously defined and the device `AnalogDevice` from `pulser.devices`. `AnalogDevice` is an example of the first generation of QPU developed at Pasqal:

In [ ]:
from pulser.devices import AnalogDevice

In [ ]:
seq = pulser.Sequence(reg, AnalogDevice)

## 3. Declaring a Channel

A `Device` implements some `Channels`, that can be used to modify the state of the atoms via the application of `Pulses`. As presented in the [conventions page](https://pulser.readthedocs.io/en/stable/conventions.html), each channel implements a different Hamiltonian. A complete description of the components of a `Channel` is provided in [its section in the Fundamentals](https://pulser.readthedocs.io/en/stable/apidoc/core.html#channels).

The `AnalogDevice` has only one `Channel`, a `Rydberg.Global` channel that can be used to perform the same single-qubit operation on all the atoms using the rydberg states.

At the beginning of the `Sequence`, all the channels of the `Device` are availble:

In [ ]:
seq.available_channels

We can see that we only have the `Rydberg.Global` channel available. To be used in the `Sequence`, we have to declare the channel and provide it a name. Note how a declared channel is no longer reported as available.

In [ ]:
seq.declare_channel("ch0", "rydberg_global")
print("Available channels after declaring 'ch0':")
pprint(seq.available_channels)

At any time, we can also consult which channels were declared, their specifications and the name they were given by calling:

In [ ]:
seq.declared_channels

## 4. Adding a Pulse

Now that we have declared a `Channel`, we can add `Pulses` to this channel. `Pulses` are defined by three quantities: the amplitude $\Omega$ (in rad/µs), the detuning $\delta$ (in rad/µs) and the phase $\phi$ (in rad).

With Pulser, we can define the value of the amplitude and detuning at each nanosecond using `Waveforms`, while the phase is constant within each pulse.

The are many ways to define a `Pulse` and its `Waveforms`. Read the [Pulse section in the Fundamentals](https://pulser.readthedocs.io/en/stable/apidoc/core.html#module-pulser.pulse) to learn more about these.

Let's program a `Pulse` that excites the atom in the Rydberg state. As described in [the white paper of Pulser](https://quantum-journal.org/papers/q-2020-09-21-327/pdf/) (see figure 7 on the `NOT` gate), a `Pulse` with $\int_0^{\Delta t} \Omega(u) du = \pi$, $\delta(t)=0$ and $\phi=0$ will perform just that.

Among the waveforms available in Pulser, the [`BlackmanWaveform`](https://pulser.readthedocs.io/en/stable/apidoc/core.html#pulser.waveforms.BlackmanWaveform) can be defined by its area under the curve. Let's define a Pulse having as amplitude a `BlackmanWaveform` of area under the curve `pi`, and having detuning and phase constant equal to 0.

In [ ]:
pi_wvf = pulser.BlackmanWaveform(600, np.pi)
print("Integral of the waveform is:", pi_wvf.integral)
pi_wvf.draw(ylabel="BlackmanWaveform of area $\\pi$")
pi_pulse = pulser.Pulse.ConstantDetuning(pi_wvf, 0, 0)

Let's add this pulse to `'ch0'`:

In [ ]:
seq.add(pi_pulse, "ch0")

The `Sequence` can be printed, and we can see that the Pulse has been added to it:

In [ ]:
print(seq)

We can also draw the sequence, for a more visual representation:

In [ ]:
seq.draw()

After this, you could add multiple `Pulses` to this channel, or define other channels to target other transitions. But let's move on to the final step: the measurement.

## 5. Measurement

To finish a sequence, we measure it. A measurement signals the end of a sequence, so after it no more changes are possible. We can measure a sequence by calling:

In [ ]:
seq.measure(basis="ground-rydberg")

When measuring, one has to select the desired measurement basis. The availabe options depend on the device and can be consulted by calling:

In [ ]:
AnalogDevice.supported_bases

We've now obtained the final sequence!

In [ ]:
seq.draw()

## 6. Submit the `Sequence` to a `Backend`

We have tailored the `Pulse` such that the atoms all go from the ground state (measurement outcome: 0) to the excited state (measurement outcome: 1) (see the [Measurement section in the "Conventions" page](https://pulser.readthedocs.io/en/stable/conventions.html#state-preparation-and-measurement)). But is it really happening ? The `Sequence` can be simulated using a [`Backend`](https://pulser.readthedocs.io/en/stable/apidoc/backend.html), a common interface for execution of Sequences using the QPU or emulators. More details about the backends are provided in [its Fundamental section](https://pulser.readthedocs.io/en/stable/tutorials/backends.html).

Prior to submitting the `Sequence` to the QPU, it is of good practice (if the number of atoms is not too big) to use an emulator to simulate the outcome of the Sequence. A list of all the available emulators is given [in the API](https://pulser.readthedocs.io/en/stable/apidoc/backend.html). Let's start with emulating the `Sequence` locally.

Pulser comes with `pulser-simulation`, a package to simulate pulser `Sequence` using the python library [**QuTip**](https://qutip.org/). It implements the `QutipBackend` class. The `Backend` is initialized with a `Sequence`, and the simulation in itself is performed when calling the method `run`.

The outcome of the `QutipBackend` for this kind of simulation is a `CoherentResult`. It stores the state of each atom in the Register at each simulation time. The state of the system at the end of the simulation can be obtained with the method `get_final_state`:


In [ ]:
sim = pulser_simulation.QutipBackend(seq)
res = sim.run()
res.get_final_state()

The final state is the rydberg state, defined by $\left|r\right> = (1, 0)$ (see the [conventions page](https://pulser.readthedocs.io/en/stable/conventions.html) for details).

On a QPU, this state cannot be accessed. We can only estimate the probability to measure the system in the ground state (measurement 0) and the excited state (measurement 1) by making multiple measurements. We can model the measurement by the QPU with the method `sample_final_state`:

In [ ]:
res.sample_final_state(N_samples=1000)

In all the measurement outcomes, all the atoms are measured in the '1' state. This means they are all always measured in the excited state: the `Sequence` works as intended !

## Follow up

- Check a detailed description of all the components of the `Sequence` introduced here in the [Fundamentals section](https://pulser.readthedocs.io/en/stable/apidoc/pulser.html) !

- In this `Sequence`, we only worked with 1 atom in the Register. Begin our tutorials with the same sequence of Pulses, applied on a Register of 2 atoms placed far appart !

- To further study the influence of an amplitude Pulse on the state of the atoms, you can have a look at our tutorial where we excite the state of the atoms in a superposition state and perform a Rabi experiment.